In [312]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import stft
from scipy.fft import fft,fftfreq

def get_fft_binned(input_waveform,min=0,max=5,sample_rate = 100,smoothing=1, bins=20):
    # SUMMARY: Computes 1-D Fast Fourier Transform on a sequence of data, returns sum between min and max frequency in Hz normalized to waveform amplitude
    # RETURNS: float (normalized total area under curve of fourier transform between min and max frequency specified)
    # LIMITATIONS: Max frequency value is 1/2 of time interval. Utilize last 4 time intervals for more accurate fourier transform on higher frequencies.
    flat = input_waveform.flatten()
    yf = fft(flat)
    xf = fftfreq(len(flat),100/len(flat))[:len(flat)//2]
    yval = 2.0/len(flat)*np.abs(yf[:len(flat)//2])
    s = pd.Series(yval)
    y = s.rolling(smoothing)
    avg = y.mean()

    bin_values, bin_edges = np.histogram(avg[(xf>min) & (xf<max)], bins=bins)
    return bin_values / bin_values.sum()
    #sns.lineplot(x=xf,y=avg,linewidth=0.5) # plot waveform
    #return avg[(xf>min) & (xf < max)].sum()/avg.sum()

def linregress(data,time):
    time_points = np.linspace(0, time, len(data))
    slope, intercept = np.polyfit(time_points, data, 1)
    return slope

def get_input_vector(data,epoch,backward=5,bins_per=50):
    #SUMMARY: Code generates an input vector given a dataset and epoch #
    upperbounds = [30,30,20,1.0,1.0]
    
    output = np.zeros(int(5*bins_per+2))
    if (epoch-5)<=0:
        return output
    for i in range(0,5):
        # fetch data subset
        subset = data[i,:,:]
        # fetch epoch subset
        subset = subset[(epoch-backward):epoch,:].flatten()
        output[bins_per*i:bins_per*(i+1)] = get_fft_binned(subset,max=upperbounds[i],bins=bins_per)
    output[5*bins_per] = np.mean(data[5,(epoch-backward):epoch,:].flatten())/40
    output[5*bins_per+1] = linregress(data[5,(epoch-backward):epoch,:].flatten(),(backward + 1)/2)*50
    return output

def get_output_vector(labels,epoch):
    # SUMMARY: Gets the output vector given an epoch #
    output = np.zeros(6)
    if (epoch-5)<=0:
        return output
    else:
        #rint(labels.shape)
        #print(epoch)
        output[int(labels[epoch]-1)]=1
        return output

In [313]:
def get_data_indices(input_data, input_labels, indices,num_per=40,bins_per=50):
    in_vecs = np.zeros((len(indices),int(5*bins_per+2)))
    out_vecs = np.zeros((len(indices),6))
    
    for i in range(0,len(indices)):
        in_vecs[i,:] = get_input_vector(input_data,indices[i],bins_per=bins_per)
        out_vecs[i,:] = get_output_vector(input_labels,indices[i])

    return in_vecs, out_vecs


def get_data_night(input_data, input_labels,num_per=40,bins_per=50):
    #SUMMARY: Gets 40 input and output vectors for each sleep state given an input data and input labels
    #RETURNS: input and ouput vectors 
    in_vectors = np.zeros((num_per*6,5*bins_per+2))
    out_vectors = np.zeros((num_per*6,6))

    for i in range(1,7):
        if(len(np.where(input_labels == i)[0])<num_per):
            indices = np.random.choice(np.where(input_labels == i)[0],size=len(np.where(input_labels == i)[0]))
        else:
            indices = np.random.choice(np.where(input_labels == i)[0],size=num_per,replace=False)
        
        while((indices<5).any()):
            indices = np.random.choice(np.where(input_labels == i)[0],size=num_per,replace=False)
        
        in_vecs, out_vecs = get_data_indices(input_data,input_labels,indices,num_per,bins_per=bins_per)
        
        
        if(len(np.where(input_labels == i)[0])<num_per):
            count = len(np.where(input_labels == i)[0])
            in_vectors[(i-1)*num_per:((i-1)*num_per+count),:] = in_vecs
            out_vectors[(i-1)*num_per:((i-1)*num_per+count),:] = out_vecs
        else:
            in_vectors[(i-1)*num_per:(i)*num_per,:] = in_vecs
            out_vectors[(i-1)*num_per:(i)*num_per,:] = out_vecs
        
    return in_vectors,out_vectors

def get_data_night_alternate(input_data, input_labels,bins_per=50,padding=20):
    # crop to data where we have > 0 for label (1 through 6 only)
    input_data = input_data[:,input_labels>0,:]
    input_labels = input_labels[input_labels>0]

    # crop to first value where input is greater than 1 and last value of same positoin
    total = len(input_labels)
    min = np.arange(total)[input_labels>1].min()
    max = np.arange(total)[input_labels>1].max()

    input_data = input_data[:,(min-20):(max+20),:]
    input_labels = input_labels[(min-20):(max+20)]

    new_total = len(input_labels)

    in_vecs, out_vecs = get_data_indices(input_data,input_labels,np.arange(new_total),bins_per=bins_per)
        
    return in_vecs,out_vecs


# Generate Testing and Training Data

We generate training data using all data from the first 21 patients (excluding p23 and p22). 

We test our data on patients 22 and 23.

In [310]:
import os

files = os.listdir("Data/")
# assume files are in order of (x,y)
len(files) # all files are seen here


86

In [311]:
num_per = 250
bins = 20
num_nights = 40

input_vectors = np.empty(num_nights,dtype=object)
output_vectors = np.empty(num_nights,dtype=object)

for i in range(0,2*num_nights,2):
    data = np.load(f"Data/{files[i]}")
    data = data.reshape((data.shape[1],data.shape[0],data.shape[2]))
    labels = np.load(f"Data/{files[i+1]}")
    input, output = get_data_night_alternate(data,labels,bins_per=bins)
    print(f"Completed Night {(i/2)}")
    input_vectors[int(i/2)] = input
    output_vectors[int(i/2)] = output

KeyboardInterrupt: 

First we will acquire training data

In [112]:
num_per = 250
bins = 30
num_nights = 40

input_vectors = np.zeros((num_nights*6*num_per,5*bins+2))
output_vectors = np.zeros((num_nights*6*num_per,6))

for i in range(0,2*num_nights,2):
    data = np.load(f"Data/{files[i]}")
    data = data.reshape((data.shape[1],data.shape[0],data.shape[2]))
    labels = np.load(f"Data/{files[i+1]}")

    input, output = get_data_night(data,labels,num_per=num_per,bins_per=bins)
    input_vectors[int(6*num_per*(i/2)):int((6*num_per*((i/2)+1))),:] = input
    output_vectors[int(6*num_per*(i/2)):int((6*num_per*((i/2)+1))),:] = output
    print(f"Completed Night {(i/2)}")

Completed Night 0.0
Completed Night 1.0
Completed Night 2.0
Completed Night 3.0
Completed Night 4.0
Completed Night 5.0
Completed Night 6.0
Completed Night 7.0
Completed Night 8.0
Completed Night 9.0
Completed Night 10.0
Completed Night 11.0
Completed Night 12.0
Completed Night 13.0
Completed Night 14.0
Completed Night 15.0
Completed Night 16.0
Completed Night 17.0
Completed Night 18.0
Completed Night 19.0
Completed Night 20.0
Completed Night 21.0
Completed Night 22.0
Completed Night 23.0
Completed Night 24.0
Completed Night 25.0
Completed Night 26.0
Completed Night 27.0
Completed Night 28.0
Completed Night 29.0
Completed Night 30.0
Completed Night 31.0
Completed Night 32.0
Completed Night 33.0
Completed Night 34.0
Completed Night 35.0
Completed Night 36.0
Completed Night 37.0
Completed Night 38.0
Completed Night 39.0


In [189]:
backup = input_vectors.copy()

In [190]:
for i in range(0,len(input_vectors)):
    input_vectors[i] = input_vectors[i][np.mean(input_vectors[i],axis=1)!=0]

In [194]:
backup2 = output_vectors.copy()

In [195]:
for i in range(0,len(output_vectors)):
    output_vectors[i] = output_vectors[i][np.mean(output_vectors[i],axis=1)!=0]

In [200]:
np.save("sequential_data_input.npy",input_vectors)

In [201]:
np.save("sequential_data_output.npy",output_vectors)

In [198]:
len(output_vectors[0])

754

In [113]:
input_vectors = input_vectors[np.mean(input_vectors,axis=1)!=0]
output_vectors = output_vectors[np.mean(output_vectors,axis=1)!=0]

In [281]:
np.save("train_X.npy",input_vectors)
np.save("train_Y.npy",output_vectors)

In [39]:
(np.mean(input_vectors,axis=1)==0).sum()

0

In [114]:
input_vectors.shape

(35838, 152)

# Training Model

In [115]:
def train_test_split(input_vecs,output_vecs,split=0.9):
    total = len(output_vecs)
    train_total = int(np.round(total*split))
    test_total = total-train_total

    indices_train = np.random.choice(int(total),size=train_total,replace=False)
    # for getting test indices
    all = pd.Series(np.arange(total))
    indices_test = all[all.isin(indices_train)==False].values

    train_X = input_vecs[indices_train,:]
    test_X = input_vecs[indices_test,:]
    
    train_Y = output_vecs[indices_train,:]
    test_Y = output_vecs[indices_test,:]

    return train_X, train_Y, test_X, test_Y
    
train_X, train_Y, test_X, test_Y = train_test_split(input_vectors,output_vectors,split=1.0)

In [10]:
train_X = np.load("train_input.npy")
train_Y = np.load("train_output.npy")

test_X = np.load("test_input.npy")
test_Y = np.load("test_output.npy")

In [12]:
train_X.shape

(4347, 252)

In [116]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#scaler.fit(train_X)
#train_X_scaled = scaler.transform(train_X)
train_X_scaled=train_X

mlp = MLPClassifier(hidden_layer_sizes=(250,100,50,30,15),
                    activation='relu',
                    solver='adam',
                    alpha=0.001,
                    learning_rate='constant',
                    learning_rate_init=0.001,
                    max_iter=1000,
                    random_state=1,
                    verbose=True,tol=0.00001,
                    n_iter_no_change=30)

mlp.fit(train_X_scaled,train_Y)

Iteration 1, loss = 2.54405496
Iteration 2, loss = 1.67558140
Iteration 3, loss = 1.43565531
Iteration 4, loss = 1.33542519
Iteration 5, loss = 1.27124212
Iteration 6, loss = 1.20863482
Iteration 7, loss = 1.16823833
Iteration 8, loss = 1.13113055
Iteration 9, loss = 1.10952048
Iteration 10, loss = 1.08833113
Iteration 11, loss = 1.06656977
Iteration 12, loss = 1.05097545
Iteration 13, loss = 1.04887798
Iteration 14, loss = 1.03769697
Iteration 15, loss = 1.01993697
Iteration 16, loss = 1.01306597
Iteration 17, loss = 1.00287208
Iteration 18, loss = 0.98709053
Iteration 19, loss = 0.98681248
Iteration 20, loss = 0.97769920
Iteration 21, loss = 0.96969929
Iteration 22, loss = 0.96296226
Iteration 23, loss = 0.95662177
Iteration 24, loss = 0.94741654
Iteration 25, loss = 0.94095047
Iteration 26, loss = 0.93611155
Iteration 27, loss = 0.92826728
Iteration 28, loss = 0.92538701


c:\Users\sanja\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(alpha=0.001, hidden_layer_sizes=(250, 100, 50, 30, 15),
              max_iter=1000, n_iter_no_change=30, random_state=1, tol=1e-05,
              verbose=True)

Next we will acquire testing data.

In [202]:
files = os.listdir("Data/")
files = files[80:86] # last 3 nights only
print(files)

num_nights = int(len(files)/2)

num_per = 250
bins = 20

input_vectors = np.empty(num_nights,dtype=object)
output_vectors = np.empty(num_nights,dtype=object)

for i in range(0,2*num_nights,2):
    data = np.load(f"Data/{files[i]}")
    data = data.reshape((data.shape[1],data.shape[0],data.shape[2]))
    labels = np.load(f"Data/{files[i+1]}")
    input, output = get_data_night_alternate(data,labels,bins_per=bins)
    print(f"Completed Night {(i/2)}")
    input_vectors[int(i/2)] = input
    output_vectors[int(i/2)] = output

['p22_n1_X.npy', 'p22_n1_y.npy', 'p22_n2_X.npy', 'p22_n2_y.npy', 'p23_n1_X.npy', 'p23_n1_y.npy']
Completed Night 0.0
Completed Night 1.0
Completed Night 2.0


In [315]:
np.load("sequential_data_input.npy",allow_pickle=True)[1].shape

(1041, 102)

In [203]:
for i in range(0,len(input_vectors)):
    input_vectors[i] = input_vectors[i][np.mean(input_vectors[i],axis=1)!=0]

for i in range(0,len(output_vectors)):
    output_vectors[i] = output_vectors[i][np.mean(output_vectors[i],axis=1)!=0]

np.save("TEST_sequential_data_output.npy",output_vectors)
np.save("TEST_sequential_data_input.npy",input_vectors)

In [117]:
num_per=250

files = os.listdir("Data/")
files = files[80:86] # last 3 nights only
print(files)

num_nights = int(len(files)/2)

num_per = 20
bins_per = 30
num_nights = 40

input_vectors = np.zeros((num_nights*6*num_per,5*bins_per+2))
output_vectors = np.zeros((num_nights*6*num_per,6))

for i in range(0,len(files),2):
    data = np.load(f"Data/{files[i]}")
    data = data.reshape((data.shape[1],data.shape[0],data.shape[2]))
    labels = np.load(f"Data/{files[i+1]}")

    input, output = get_data_night(data,labels,num_per=num_per,bins_per=bins)
    input_vectors[int(6*num_per*(i/2)):int((6*num_per*((i/2)+1))),:] = input
    output_vectors[int(6*num_per*(i/2)):int((6*num_per*((i/2)+1))),:] = output
    print(f"Completed Night {(i/2)}")

input_vectors = input_vectors[np.mean(input_vectors,axis=1)!=0]
output_vectors = output_vectors[np.mean(output_vectors,axis=1)!=0]

test_X = input_vectors
test_Y = output_vectors

['p22_n1_X.npy', 'p22_n1_y.npy', 'p22_n2_X.npy', 'p22_n2_y.npy', 'p23_n1_X.npy', 'p23_n1_y.npy']
Completed Night 0.0
Completed Night 1.0
Completed Night 2.0


In [118]:
#test_X_scaled = scaler.transform(test_X)
test_X_scaled = test_X
mlp.score(test_X_scaled,test_Y)

0.4171974522292994

In [301]:
np.save("test_input.npy",test_X)
np.save("train_input.npy",train_X)
np.save("test_output.npy",test_Y)
np.save("train_output.npy",train_Y)

In [119]:
correct = np.zeros(6)
incorrect = np.zeros(6)

for i in range(0,len(test_X_scaled)):
    predictedprob = mlp.predict_proba(test_X_scaled[i,:].reshape(1,-1))
    predicted = (predictedprob==np.max(predictedprob)).astype(np.uint8)
    #print(predicted)
    if((test_Y[i].astype(np.int32)==predicted[0]).all()):
        correct = correct + test_Y[i]
    else:
        incorrect = incorrect + test_Y[i]

print(correct/(correct+incorrect))

[0.86666667 0.4        0.76666667 0.17307692 0.         0.35      ]


In [120]:
np.sum(train_Y,axis=0)

array([10000.,  2935.,  9967.,  3244.,  2256.,  7436.])

In [121]:
np.sum(correct)/(np.sum(correct+incorrect))

0.4840764331210191

In [173]:
np.save('input_vectors_noncorrupted_even_more.npy',input_vectors)

In [174]:
np.save('output_vectors_noncorrupted_even_more.npy',output_vectors)

In [222]:
# N x 721 x 102
output = np.zeros((3,721,102))
for i in range(0,len(input_vectors)):
    output[i,:,:] = input_vectors[i][0:721,:]

np.save("test_input_3d.npy",output)

# N x 721 x 102
out2 = np.zeros((3,721,6))
for i in range(0,len(output_vectors)):
    out2[i,:,:] = output_vectors[i][0:721,:]

np.save("test_output_3d.npy",out2)

out2[0]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [295]:
input_vectors = np.load("sequential_data_input.npy",allow_pickle=True)
output_vectors = np.load("sequential_data_output.npy",allow_pickle=True)

sizes = np.zeros(40)
for i in range(0,40):
    sizes[i] = input_vectors[i].shape[0]

input_vectors = input_vectors[sizes>720]
output_vectors = output_vectors[sizes>720]


# N x 721 x 102
output = np.zeros((38,721,102))
for i in range(0,len(input_vectors)):
    output[i,:,:] = input_vectors[i][0:721,:]

np.save("train_input_3d.npy",output)

# N x 721 x 102
out2 = np.zeros((38,721,6))
for i in range(0,len(output_vectors)):
    out2[i,:,:] = output_vectors[i][0:721,:]

np.save("train_output_3d.npy",out2)

In [296]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#scaler.fit(train_X)
#train_X_scaled = scaler.transform(train_X)
train_X = output.reshape((38,721*102))
train_Y = out2.reshape((38,721*6))

mlp = MLPClassifier(hidden_layer_sizes=(1024,512),
                    activation='relu',
                    solver='adam',
                    alpha=0.001,
                    learning_rate='adaptive',
                    learning_rate_init=0.001,
                    max_iter=30,
                    random_state=1,
                    verbose=True,tol=0.00001,
                    n_iter_no_change=30)

mlp.fit(train_X,train_Y)

Iteration 1, loss = 3005.32435585
Iteration 2, loss = 2713.37875579
Iteration 3, loss = 2242.78840589
Iteration 4, loss = 1909.87977123
Iteration 5, loss = 1728.79735169
Iteration 6, loss = 1633.02751246
Iteration 7, loss = 1601.46453560
Iteration 8, loss = 1587.81876251
Iteration 9, loss = 1572.97297591
Iteration 10, loss = 1554.08844651
Iteration 11, loss = 1530.23430250
Iteration 12, loss = 1505.89927193
Iteration 13, loss = 1479.59183885
Iteration 14, loss = 1453.90687737
Iteration 15, loss = 1425.74610036
Iteration 16, loss = 1397.39497195
Iteration 17, loss = 1366.75834675
Iteration 18, loss = 1333.41497317
Iteration 19, loss = 1298.16961201
Iteration 20, loss = 1261.97382098
Iteration 21, loss = 1226.73969513
Iteration 22, loss = 1192.51166413
Iteration 23, loss = 1158.64710132
Iteration 24, loss = 1125.91075166
Iteration 25, loss = 1093.03021467
Iteration 26, loss = 1058.83954738
Iteration 27, loss = 1023.78902573
Iteration 28, loss = 988.08951630
Iteration 29, loss = 950.99820

c:\Users\sanja\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.001, hidden_layer_sizes=(1024, 512),
              learning_rate='adaptive', max_iter=30, n_iter_no_change=30,
              random_state=1, tol=1e-05, verbose=True)

In [230]:
test_X = np.load("test_input_3d.npy").reshape(3,721*102)
test_Y = np.load("test_output_3d.npy").reshape(3,721*6)

In [293]:
for i in range(0,len(test_X)):
    predicted = mlp.predict_proba(test_X[i].reshape(1, -1))
    
    reshaped = predicted.reshape((721,6))
    # take max
    output = np.zeros((721,6))
    for j in range(0,721):
        output[j] = reshaped[j]==np.max(reshaped[j])
    success = 0
    fail = 0
    truth = test_Y[i].reshape((721,6))

    for j in range(0,721):
        if((output[j,:]==truth[j,:]).all()):
            success = success + 1
        else:
            fail = fail + 1
    print(success/(success+fail))

0.3079056865464632
0.46601941747572817
0.5339805825242718
